# <center> KIT Praktikum NN: L2-regularized Logistic Least Squares Regression </center>

</br>
On this exercise, you are going to apply what you learn from the `numpy` tutorial in the implementation of L2-regularized Logistic Least Squares Regression (LLSR). I will provide you the formula by now (you can do it yourself after the next lecture!!!), first you should use pens and papers to vectorize them. Then you implement the full of the classifier based on your vectorized version.

<img src="../Images/LogisticRegression.png" style="width:298px;height:275px">

</br>
L2-regularized Logistic Least Squares Regression is similar to the standard Logistic Regression: It is a binary classifier containing only one layer, mapping the input features to only one output using sigmoid function. The differents here are two things: 
* Instead of the _binary crossentropy error_ for the loss, it uses the _squared error_.
* It is applied the L2-regularization.

Note that we will do an SGD training for this exercise. More specifically:
* There are $m$ data instance on the training set, each has $n$ input features. 
* $x_{i}^{(j)}$ denotes the $i^{th}$ input feature of the $j^{th}$ data instance.
* $y^{(j)}$ denotes the binary label ($0$ or $1$) of the $j^{th}$ data instance.
* $w_{i}$ denotes the weight connecting the $i^{th}$ input feature to the output.
* $b$ is the bias of the Logistic Least Squares Regression.

So the steps of an unvectorized version are:
* The weights are initialized using Xavier Initialization, the bias can be initialized as 0.
* Train over 5 epochs, each epoch we do those steps:
  *  Loop over every data instance $x^{(j)}$:
     * Calculate the output of the LLSR: $o^{(j)} = \sigma(\sum_{i=1}^{n} w_ix_i^{(j)} + b)$
     * Calculate the cost: squared error $c^{(j)} = (y^{(j)} - o^{(j)})^2$
     * The final loss function is L2-regularized: $l^{(j)} = \frac{1}{2}c^{(j)} + \frac{\lambda}{2}\sum_{i=1}^{n}w_i^2$. 
     * Update the weights: 
         * Loop over every weight $w_i$ and update once at a time: $w_i = w_i - \eta((o^{(j)}-y^{(j)})o^{(j)}(1-o^{(j)})x_i^{(j)} + \lambda w_i)$
     * Update the bias: $b = b - \eta (o^{(j)}-y^{(j)})o^{(j)}(1-o^{(j)})$
  *  Calculate the total loss (of the epoch): $L = \frac{1}{m}\sum_{j=1}^{m}l^{(j)}$. Print it out. 


The guideline is to avoid explicit for-loops. _Hint_: We cannot make the epoch loop disappears, but all other loops can be replaced by vectorization.

First, let's import numpy and math:

In [1]:
import numpy as np
import math


We will use LLSR for the MNIST_SEVEN task: predict a $128\times 128$-pixel image of a handwritten digit whether it is a "7" or not. This is a binary classification task. I did the data reading for you. There is 5000 images, I split the first 4000 images for training, 500 images for tuning, 500 images for test. On this exercise we do not need to tune anything, so we'd leave the tuning (called the _dev set_) untouch. The first field is the label ("0"-"9") of the image, the rest are the grayscale value of each pixel. 

In [2]:
data_path = "../Data/mnist_seven.csv"
data = np.genfromtxt(data_path, delimiter=",", dtype="uint8")
train, dev, test = data[:4000], data[4000:4500], data[4500:]

In [3]:
def normalize(dataset):
    X = dataset[:, 1:] / 255.     # Normalize input features
    Y = (dataset[:, 0] == 7) * 1  # Convert labels from 0-9 to Is7 (1) or IsNot7(0)
    return X.T,Y.reshape(1, -1)

In [4]:
X_train, Y_train = normalize(train)
print(X_train.shape)
print(Y_train.shape)

X_test, Y_test = normalize(test)
print(X_test.shape)
print(Y_test.shape)

# shuffle the training data since we do SGD
# we shuffle outside the training 
# since we want to compare unvectorized and vectorized versions
# It doesn't affect to batch training later
np.random.seed(8888)     # Do not change those seedings to make our results comparable
np.random.shuffle(train) 


(784, 4000)
(1, 4000)
(784, 500)
(1, 500)


# Unvectorized Version of Stochastic Gradient Descent

First the unvectorized version of training:

In [5]:
def train_unvectorized(X_train, Y_train, lr=0.2, lambdar=0.0001, epochs=5):
    
    n = X_train.shape[0]
    m = X_train.shape[1]
    
    # Xavier Initialization
    np.random.seed(1234)
    w = np.random.randn(n) * (np.sqrt(2. / (n + 1)))
    b = 0

    for epoch in range(epochs):
        L = 0
        for j in range(m):   # Loop over every training instance
            # Forward pass
            # CODE HERE
            out = b
            
            for i in range(len(w)):
                out += w[i]*X_train[i,j]
            
            o = 1/(1 + np.exp(-out))

            # Calculate the loss
            # CODE HERE
            L += (Y_train[0,j] - o)**2
            
            # Backward pass and update the weights/bias
            # CODE HERE
            for i in range(len(w)):
                w[i] -= lr*((o - Y_train[0,j])*o*(1 - o)*X_train[i,j] + lambdar*w[i])
                
            b -= lr*((o - Y_train[0,j])*o*(1 - o))
        
        # Accumulate the total loss and print it
        L /= m
        print("Error of the epoch {0}: {1}".format(epoch + 1, L))
    
    return w, b
        

And the (unvectorized) inference:

In [6]:
def test_unvectorized(X_test, Y_test, w, b):
    
    n_test = X_test.shape[0]
    m_test = X_test.shape[1]
    corrects = 0
    
    for j in range(m_test):
        
        # Forward pass
        # CODE HERE
        out = b
            
        for i in range(len(w)):
            out += w[i]*X_test[i,j]

        o = 1/(1 + np.exp(-out))
        
        # Evaluate the outputs
        # CODE HERE
        if np.rint(o) == Y_test[0,j]:
            corrects += 1
        
        
    print("Accuracy of our LLSR:" + str((corrects * 100.) / m_test) + "%")
    
    return corrects


Test on our test data. The accuracy should be better than 89.2%. This high score 89.2% is the baseline, achieved by do nothing rather than predicting all images are not a "seven" :p.

In [7]:
w, b = train_unvectorized(X_train, Y_train)
_ = test_unvectorized(X_test, Y_test, w, b)

Error of the epoch 1: 0.02227099791815772
Error of the epoch 2: 0.015494194498583323
Error of the epoch 3: 0.013894081012307198
Error of the epoch 4: 0.012974585494809749
Error of the epoch 5: 0.012167139245510663
Accuracy of our LLSR:98.4%


# Vectorized Version of Stochastic Gradient Descent

Now we move to the vectorized version of training and inference, just replace for-loops and total-sums by $np.dot()$,  $np.sum()$ and the numpy pair-wise operations (you should do the vectorization using pens and papers first).

In [8]:
def train_vectorized(X_train, Y_train, lr=0.2, lambdar=0.0001, epochs=5):
    
    n = X_train.shape[0]
    m = X_train.shape[1]
    
    # Xavier Initialization
    np.random.seed(1234)
    w = np.random.randn(n) * (np.sqrt(2. / (n + 1)))
    b = 0

    for epoch in range(epochs):
        L = 0
        for j in range(m):

            # Forward pass
            # CODE HERE
            out = b            
            out += np.dot(w, X_train[:,j])            
            o = 1/(1 + np.exp(-out))            
            
            # Calculate the loss (for each instance - SGD) 
            # CODE HERE
            L += (Y_train[0,j] - o)**2
            
            # Backward pass and update the weights/bias (for each instance - SGD) 
            # CODE HERE
            w -= lr*((o - Y_train[0,j])*o*(1 - o)*X_train[:,j] + lambdar*w)                
            b -= lr*((o - Y_train[0,j])*o*(1 - o))
            
        L /= m
        print("Error of the epoch {0}: {1}".format(epoch + 1, L))
    return w, b

And the vectorized inference (short, clear and fast):

In [9]:
def test_vectorized(X_test, Y_test, w, b):
    
    m_test = X_test.shape[1]
    
    
    out = b
    out += np.dot(w, X_test)
    
    o = 1/(1 + np.exp(-out))
    
    corrects = np.sum(np.rint(o) == Y_test)
    
    print("Accuracy of our LLSR:" + str((corrects * 100.) / m_test) + "%")
    
    return corrects


Those following runs should return exact the same outputs like the (unvectorized) training and inference before but in less than a second. The vectorization should be more effective (much faster) if this is not an one-layer logistic regression but a deep network.

In [10]:
w, b = train_vectorized(X_train, Y_train)
_ = test_vectorized(X_test, Y_test, w, b)

Error of the epoch 1: 0.022270997918157717
Error of the epoch 2: 0.015494194498583325
Error of the epoch 3: 0.013894081012307201
Error of the epoch 4: 0.012974585494809752
Error of the epoch 5: 0.012167139245510668
Accuracy of our LLSR:98.4%


# Vectorized Version of Batch Gradient Descent 

Here is the fully vectorized version, batch training (vectorizing over training instances). The formula (you might be able to derive them after the next lecture):

$$ z = w \cdot X + b $$

$$ o = \sigma(z) $$

$$ C = \frac{1}{2m}\sum_{j=1}^{m}(y^{(j)}-o^{(j)})^2 $$

$$ R = \frac{1}{2m}\sum_{i=1}^{n}w_i^2 $$

$$ L = C + \lambda R $$

$$ \frac{\partial C}{\partial z^{(j)}} = \frac{1}{m}(o^{(j)} - Y^{(j)}) * o^{(j)} * (1 - o^{(j)}) $$

$$ \frac{\partial z^{(j)}}{\partial w_i} = x_i $$

$$ \Rightarrow \frac{\partial C}{\partial w} = \frac{\partial C}{\partial z} \cdot X^T $$

$$ \frac{\partial R}{\partial w} = \frac{1}{m}w $$ 

$$ \Rightarrow \frac{\partial L}{\partial w} = \frac{\partial C}{\partial w} + \lambda\frac{\partial R}{\partial w} $$

$$ \frac{\partial z}{\partial b} = 1 $$

$$ \Rightarrow \frac{\partial L}{\partial b} = \frac{\partial C}{\partial b} = \frac{1}{m} \sum_{j=1}^{m}(o^{(j)} - Y^{(j)}) * o^{(j)} * (1 - o^{(j)}) $$

$$ w = w - \eta * \frac{\partial L}{\partial w} $$

$$ b = b - \eta *  \frac{\partial L}{\partial b} $$

In [27]:
def train_batch(X_train, Y_train, lr=0.1, lambdar=0.0001, epochs=50):
    
    n = X_train.shape[0]
    m = X_train.shape[1]

    # Xavier Initialization
    np.random.seed(1234)
    w = np.random.randn(1, n) * (np.sqrt(2. / (n + 1)))
    b = 0

    for epoch in range(epochs):

        # Forward pass
        # CODE HERE
        z = b
            
        #for i in range(len(w)):
            #out += w[i]*X_train[i,j]
        z += np.dot(w, X_train)

        o = 1.0/(1.0 + np.exp(-z))
        #o = np.exp(z) / (1+np.exp(z))
        
        C = np.sum((Y_train - o)**2)/(2*m)
        
        R = np.sum(w**2)/(2*m)

        # Calculate the loss 
        # CODE HERE
        L = C + lambdar*R
        
        # Backward pass and update the weights/bias
        # CODE HERE
        dCdZ = ((o - Y_train) * o * (1 - o))/(m)
        dCdw = np.dot(dCdZ, X_train.T)
        dRdw = w/m
        dLdw = dCdw + lambdar*dRdw
        
        dLdb = (np.sum((o - Y_train) * o * (1 - o)))/m
        
        w -= lr*dLdw
        b -= lr*dLdb
        
        print("Error of the epoch {0}: {1}".format(epoch + 1, L))
        
    return w, b
        

Since it is a batch training and requires different hyperparameters, the result might not be comparable to the SGD trainings above. 

In [28]:
w_batch, b_batch = train_batch(X_train, Y_train, lr=2, lambdar=0.5, epochs=1000)
_ = test_vectorized(X_test, Y_test, w_batch, b_batch)

Error of the epoch 1: 0.1678707497382271
Error of the epoch 2: 0.05099793891240988
Error of the epoch 3: 0.05099029685926981
Error of the epoch 4: 0.05098236124768441
Error of the epoch 5: 0.05097411387531518
Error of the epoch 6: 0.05096553499098012
Error of the epoch 7: 0.05095660312598227
Error of the epoch 8: 0.05094729490290709
Error of the epoch 9: 0.05093758481830059
Error of the epoch 10: 0.05092744499497388
Error of the epoch 11: 0.05091684489886967
Error of the epoch 12: 0.050905751014439275
Error of the epoch 13: 0.050894126471271034
Error of the epoch 14: 0.050881930613224616
Error of the epoch 15: 0.05086911849948958
Error of the epoch 16: 0.05085564032470598
Error of the epoch 17: 0.050841440742436905
Error of the epoch 18: 0.050826458072708164
Error of the epoch 19: 0.0508106233698151
Error of the epoch 20: 0.050793859320861404
Error of the epoch 21: 0.05077607893816012
Error of the epoch 22: 0.05075718399918558
Error of the epoch 23: 0.05073706317552298
Error of the epo

Error of the epoch 204: 0.0084048970357069
Error of the epoch 205: 0.008394994120004582
Error of the epoch 206: 0.008385175423975152
Error of the epoch 207: 0.00837543964114884
Error of the epoch 208: 0.008365785492687981
Error of the epoch 209: 0.008356211726643343
Error of the epoch 210: 0.008346717117235008
Error of the epoch 211: 0.008337300464156757
Error of the epoch 212: 0.008327960591903142
Error of the epoch 213: 0.008318696349118283
Error of the epoch 214: 0.00830950660796566
Error of the epoch 215: 0.008300390263517988
Error of the epoch 216: 0.00829134623316653
Error of the epoch 217: 0.008282373456049015
Error of the epoch 218: 0.008273470892495565
Error of the epoch 219: 0.008264637523491858
Error of the epoch 220: 0.008255872350158993
Error of the epoch 221: 0.008247174393249367
Error of the epoch 222: 0.008238542692658026
Error of the epoch 223: 0.008229976306948905
Error of the epoch 224: 0.008221474312895431
Error of the epoch 225: 0.008213035805034965
Error of the ep

Error of the epoch 417: 0.00721262693169843
Error of the epoch 418: 0.007209162316238089
Error of the epoch 419: 0.007205708060666927
Error of the epoch 420: 0.007202264107065644
Error of the epoch 421: 0.00719883039816793
Error of the epoch 422: 0.007195406877351826
Error of the epoch 423: 0.007191993488631229
Error of the epoch 424: 0.007188590176647492
Error of the epoch 425: 0.007185196886661125
Error of the epoch 426: 0.007181813564543607
Error of the epoch 427: 0.007178440156769285
Error of the epoch 428: 0.007175076610407399
Error of the epoch 429: 0.007171722873114169
Error of the epoch 430: 0.007168378893125014
Error of the epoch 431: 0.007165044619246838
Error of the epoch 432: 0.007161720000850429
Error of the epoch 433: 0.007158404987862942
Error of the epoch 434: 0.007155099530760473
Error of the epoch 435: 0.007151803580560716
Error of the epoch 436: 0.007148517088815726
Error of the epoch 437: 0.0071452400076047525
Error of the epoch 438: 0.00714197228952716
Error of the

Error of the epoch 641: 0.006628545492893752
Error of the epoch 642: 0.0066266040001352125
Error of the epoch 643: 0.006624667147242654
Error of the epoch 644: 0.0066227349207806105
Error of the epoch 645: 0.006620807307353225
Error of the epoch 646: 0.006618884293604126
Error of the epoch 647: 0.006616965866216331
Error of the epoch 648: 0.006615052011912138
Error of the epoch 649: 0.006613142717453013
Error of the epoch 650: 0.006611237969639499
Error of the epoch 651: 0.006609337755311124
Error of the epoch 652: 0.006607442061346298
Error of the epoch 653: 0.006605550874662219
Error of the epoch 654: 0.006603664182214798
Error of the epoch 655: 0.006601781970998559
Error of the epoch 656: 0.006599904228046558
Error of the epoch 657: 0.0065980309404303095
Error of the epoch 658: 0.006596162095259694
Error of the epoch 659: 0.006594297679682887
Error of the epoch 660: 0.006592437680886284
Error of the epoch 661: 0.006590582086094419
Error of the epoch 662: 0.006588730882569903
Error o

Error of the epoch 832: 0.00632853219288348
Error of the epoch 833: 0.006327274487655678
Error of the epoch 834: 0.006326019491016245
Error of the epoch 835: 0.006324767195714594
Error of the epoch 836: 0.006323517594523742
Error of the epoch 837: 0.006322270680240193
Error of the epoch 838: 0.006321026445683853
Error of the epoch 839: 0.0063197848836979206
Error of the epoch 840: 0.006318545987148794
Error of the epoch 841: 0.006317309748925973
Error of the epoch 842: 0.006316076161941956
Error of the epoch 843: 0.006314845219132146
Error of the epoch 844: 0.006313616913454751
Error of the epoch 845: 0.00631239123789069
Error of the epoch 846: 0.00631116818544349
Error of the epoch 847: 0.006309947749139193
Error of the epoch 848: 0.006308729922026263
Error of the epoch 849: 0.0063075146971754825
Error of the epoch 850: 0.006306302067679858
Error of the epoch 851: 0.006305092026654535
Error of the epoch 852: 0.006303884567236685
Error of the epoch 853: 0.006302679682585427
Error of th

One thing to compare: the speed. Try to run the same number of epochs (1000) with SGD, vectorized training, you can see it still takes a long time to run compared to the fully batch training.

In [ ]:
w, b = train_vectorized(X_train, Y_train, epochs=1000)
_ = test_vectorized(X_test, Y_test, w, b)

# Vectorized Version of Minibatch Gradient Descent

Finally, we can do minibatch training, it is the same as batch training (the formula) but one iteration runs over a subset of the whole dataset at a time, and those subsets (minibatches) are shuffle before training:

In [31]:
def train_minibatch(X_train, Y_train, batch_size=256, lr=0.1, lambdar=0.0001, epochs=50):
    
    n = X_train.shape[0]
    
    # Xavier Initialization
    np.random.seed(1234)
    w = np.random.randn(1, n) * (np.sqrt(2. / (n + 1)))
    b = 0

    for epoch in range(epochs):
        
        # Split into minibatches 
        # CODE HERE
        X_minibatches = np.array_split(X_train, batch_size, axis=1)
        Y_minibatches = np.array_split(Y_train, batch_size, axis=1)
        
        # We shuffle the minibatches of X and Y in the same way
        # CODE HERE
        perm = np.random.permutation(len(X_minibatches))
        
        # Now we can do the training, we cannot vectorize over different minibatches
        # They are like our "epochs"
        for i in range(len(X_minibatches)): # CODE HERE
            
            # Extract a minibatch to do training
            X_current = X_minibatches[perm[i]] # CODE HERE
            Y_current = Y_minibatches[perm[i]] # CODE HERE
            m = X_current.shape[1]
            
            z = np.dot(w, X_current) + b

            o = 1.0/(1.0 + np.exp(-z))

            C = np.sum((Y_current - o)**2)/(2*m)

            R = np.sum(w**2)/(2*m)

            # Calculate the loss 
            # CODE HERE
            L = C + lambdar*R

            # Backward pass and update the weights/bias
            # CODE HERE
            dCdZ = ((o - Y_train) * o * (1 - o))/(m)
            dCdw = np.dot(dCdZ, X_current.T)
            dRdw = w/m
            dLdw = dCdw + lambdar*dRdw

            dLdb = (np.sum((o - Y_current) * o * (1 - o)))/m

            w -= lr*dLdw
            b -= lr*dLdb

            print("Error of the iteration {0}: {1}".format(epoch * B + i + 1, L))

    return w, b

Minibatch Training for this LLSR is very sensitive to hyperparameter choosing. Should use with early stopping. Do not supprise if the accurary is bad. Shuffling the minibatch also takes time, so do not run this with large number of epochs.

In [30]:
# Do not run this for more than 100 epochs!!!!!!!!!
w_minibatch, b_minibatch = train_minibatch(X_train, Y_train, batch_size=512, lr=0.001, lambdar=0.0001, epochs=30)
_ = test_vectorized(X_test, Y_test, w_minibatch, b_minibatch)

ValueError: shapes (1,784) and (1,4000) not aligned: 784 (dim 1) != 1 (dim 0)